In [16]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.nn import softmax

In [6]:
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train,x_test = (x_train/255.0),(x_test/255.0)

11493376/11490434 [==============================] - 1s 0us/step


In [12]:
model = Sequential([
    Flatten(input_shape=(28,28)),
    Dense(128,activation='relu'),
    Dropout(0.2),
    Dense(128,activation='relu'),
    Dropout(0.3),
    Dense(128,activation='relu'),
    Dropout(0.2),
    Dense(10)
])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0

In [15]:
prediction = model.predict(x_train[:1])
prediction

array([[-0.13078469,  0.07431459,  0.14502199,  0.36558002,  0.02645301,
        -0.12695244,  0.0602425 ,  0.25842288, -0.01641271, -0.20656022]],
      dtype=float32)

In [18]:
softmax(prediction).numpy()

array([[0.08266793, 0.10148703, 0.10892271, 0.1358018 , 0.09674412,
        0.08298535, 0.1000689 , 0.12200223, 0.09268473, 0.07663519]],
      dtype=float32)

In [19]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [22]:
loss_function(y_train[:1], prediction).numpy()

2.4890912

In [24]:
model.compile(optimizer='adam',loss=loss_function,metrics=['accuracy'])

In [28]:
model.fit(x_train,y_train,batch_size=128,epochs=5)

Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 0.0474 - accuracy: 0.9848
Epoch 2/5
469/469 [==============================] - 2s 3ms/step - loss: 0.0444 - accuracy: 0.9858
Epoch 3/5
469/469 [==============================] - 2s 3ms/step - loss: 0.0433 - accuracy: 0.9860
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0445 - accuracy: 0.9855
Epoch 5/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0409 - accuracy: 0.9870


In [29]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0698 - accuracy: 0.9813


[0.06975351274013519, 0.9812999963760376]

In [30]:
model_with_softmax = Sequential([
                                 model,
                                 tf.keras.layers.Softmax()
                                 ])

In [33]:
model_with_softmax(x_test[:5]).numpy()

array([[1.36198046e-13, 6.39881551e-12, 8.69406502e-09, 1.15299038e-07,
        8.43943145e-12, 1.67419949e-11, 2.88516065e-17, 9.99999881e-01,
        1.38546509e-12, 2.09039293e-08],
       [3.39666663e-13, 2.93504776e-10, 1.00000000e+00, 3.69439271e-11,
        1.30269951e-15, 1.13058002e-14, 7.55645614e-14, 7.69221631e-11,
        4.55946533e-13, 7.04464621e-20],
       [3.66355085e-10, 9.99998927e-01, 6.58290560e-08, 1.51421542e-10,
        9.59083195e-08, 1.97473704e-09, 2.54553157e-07, 4.87249139e-08,
        5.81683707e-07, 7.20660531e-10],
       [9.99979138e-01, 1.03954276e-10, 2.81988287e-06, 2.16209557e-08,
        2.33744384e-08, 1.88277554e-07, 1.70720123e-05, 9.74676482e-08,
        2.83596719e-08, 5.39854227e-07],
       [2.62794828e-07, 6.61005174e-07, 2.75530340e-07, 1.08566423e-09,
        9.99385476e-01, 1.41699505e-08, 9.74090426e-08, 6.04840398e-06,
        3.97085431e-09, 6.07157010e-04]], dtype=float32)

In [34]:
print("Training Finished")

Training Finished


In [36]:
model.save_weights("tf_model.ckpt")

In [39]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_with_softmax)

In [40]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_model = converter.convert()

In [41]:
open("mnist_with_softmax.tflite", "wb").write(quantized_model)

138312

In [43]:
from google.colab import files

In [44]:
files.download("mnist_with_softmax.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>